In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from neural_network import NeuralNetworkModel, ChessDataset, ChessEvaluationNeuralNetwork

In [4]:
def eval_to_output(eval: str):
    if eval.startswith('#+'):
        return 10000.0
    elif eval.startswith('#-'):
        return -10000.0
    else:
        return float(eval)

In [5]:
df = pd.read_csv("../../data/chess_data.csv")

X = df['FEN'].apply(NeuralNetworkModel.fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(eval_to_output).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [6]:
model = ChessEvaluationNeuralNetwork(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 935961.3182
Epoch 2, Loss: 871773.0489
Epoch 3, Loss: 819744.6439
Epoch 4, Loss: 782289.5968
Epoch 5, Loss: 754557.6215
Epoch 6, Loss: 730450.8218
Epoch 7, Loss: 709485.6755
Epoch 8, Loss: 692245.3581
Epoch 9, Loss: 679561.2578
Epoch 10, Loss: 663387.7985
Epoch 11, Loss: 643700.2670
Epoch 12, Loss: 625698.2033
Epoch 13, Loss: 614728.5343
Epoch 14, Loss: 589871.0330
Epoch 15, Loss: 572128.2108
Epoch 16, Loss: 562436.8250
Epoch 17, Loss: 550468.7705
Epoch 18, Loss: 530122.7171
Epoch 19, Loss: 514406.4097
Epoch 20, Loss: 506651.8913
Epoch 21, Loss: 478362.5695
Epoch 22, Loss: 477702.8888
Epoch 23, Loss: 453277.0479
Epoch 24, Loss: 457292.1284
Epoch 25, Loss: 438992.2032
Epoch 26, Loss: 435118.6607
Epoch 27, Loss: 421741.4523
Epoch 28, Loss: 418006.8311
Epoch 29, Loss: 406094.1141
Epoch 30, Loss: 400271.3248
Epoch 31, Loss: 398390.8109
Epoch 32, Loss: 385688.5066
Epoch 33, Loss: 377239.1786
Epoch 34, Loss: 370701.5707
Epoch 35, Loss: 358588.6721
Epoch 36, Loss: 362448.3659
E

In [7]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 731.1052


In [8]:
torch.save(model.state_dict(), 'neural_network.pth')